In [2]:
import pyarrow.dataset as ds
import duckdb
import plotly.express as px
import json

from plotly.graph_objs import Figure
import numpy as np

DIMENSIONS = {
    "Região geográfica": "CO_REGIAO",
    "Sigla da Unidade da Federeção": "SG_UF",
    "Unidade da Federação": "CO_UF",
    "Mesorregião": "CO_MESORREGIAO",
    "Microrregião": "CO_MICRORREGIAO",
    "Município": "NO_MUNICIPIO"
}

con = duckdb.connect('../file.db')

microdados = ds.dataset("../data/transformed/microdados.parquet", format="parquet", partitioning="hive")
con.register("microdados", microdados)

# indicadores = {}
# for indicador in INDICADORES:
#     indicadores[indicador] = ds.dataset(f"../data/transformed/indicadores/{indicador}.parquet", format="parquet", partitioning="hive")
#     con.register(indicador, indicadores[indicador])

In [38]:
dimension = "Município"
df = con.execute(f"""
                select
                    NU_ANO_CENSO as 'Ano',
                    CO_MUNICIPIO as 'Código do Município',
                    NO_MUNICIPIO as 'Nome do Município',
                    round(cast(count(*) filter (where IN_AGUA_REDE_PUBLICA) as float) / count(*) * 100, 2) as 'Nível de acesso em %',
                    --round(cast(count(*) filter (where IN_AGUA_POCO_ARTESIANO) as float) / count(*), 3) as 'Poço artesiano',
                    --round(cast(count(*) filter (where IN_AGUA_CACIMBA) as float) / count(*), 3) as 'Cacimba/Cisterna/Poço',
                    --round(cast(count(*) filter (where IN_AGUA_FONTE_RIO) as float) / count(*), 3) as 'Fonte/Rio/Igarapé/Riacho/Córrego',
                    --round(cast(count(*) filter (where IN_AGUA_INEXISTENTE) as float) / count(*), 3) as 'Não há abastecimento de água',
--                     round(
--                         cast(count(*) filter
--                             (where not (IN_AGUA_REDE_PUBLICA or IN_AGUA_POCO_ARTESIANO or IN_AGUA_CACIMBA or IN_AGUA_FONTE_RIO or IN_AGUA_INEXISTENTE ))
--                             as float)
--                         / count(*)
--                     , 3) as 'Sem informações'
                from microdados
                group by 1, 2, 3
                order by 1, 2, 3
            """
).df()

In [39]:
df

,Ano,Código do Município,Nome do Município,Nível de acesso em %
0,2016,1100015,Alta Floresta D'Oeste,13.110000
1,2016,1100023,Ariquemes,40.000000
2,2016,1100031,Cabixi,44.439999
3,2016,1100049,Cacoal,56.060001
4,2016,1100056,Cerejeiras,55.560001
...,...,...,...,...
38985,2022,5222005,Vianópolis,92.860001
38986,2022,5222054,Vicentinópolis,100.000000
38987,2022,5222203,Vila Boa,71.430000
38988,2022,5222302,Vila Propício,71.430000


In [52]:
def transform_geojson(geojson: dict) -> dict:
    features = geojson["features"]
    transformed_features = []
    for feature in features:
        properties = feature["properties"]
        properties["GEOCODIGO"] = int(properties["GEOCODIGO"])

        transformed_features.append(
            {
                "type": feature["type"],
                "geometry": feature["geometry"],
                "properties": properties
            }
        )

        geojson["features"] = features
    return geojson

for geoname in ["uf", "municipio", "microrregiao", "mesorregiao"]:
    with open(f"../data/geo/{geoname}.json", encoding="latin1") as f:
        geojson = json.load(f)

    geojson = transform_geojson(geojson)
    with open(f"../data/geo/{geoname}.json", "w") as file:
        json.dump(geojson, file)





In [6]:
with open(f"../data/geo/uf.json", encoding="latin1") as f:
        geojson = json.load(f)

In [11]:
[a["properties"]["NOME_UF"] for a in geojson["features"]]

['Acre',
 'Alagoas',
 'Amazonas',
 'Amapá',
 'Bahia',
 'Ceará',
 'DF',
 'Espírito Santo',
 'Goiás',
 'Maranhão',
 'Minas Gerais',
 'Mato Grosso do Sul',
 'Mato Grosso',
 'Pará',
 'Paraíba',
 'Pernambuco',
 'Piauí',
 'Paraná',
 'Rio de Janeiro',
 'Rio Grande do Norte',
 'Rondônia',
 'Roraima',
 'Rio Grande do Sul',
 'Santa Catarina',
 'Sergipe',
 'São Paulo',
 'Tocantins']

In [45]:
fig = px.choropleth_mapbox(
    df,
    geojson=geojson,
    color="Nível de acesso em %",
    locations=df.columns[2],
    range_color=(0, 100),
    mapbox_style = "white-bg",
    #featureidkey="properties.NOME",
    featureidkey="properties.GEOCODIGO",
    center = {"lat": -12, "lon": -53},
    #animation_frame="Ano",
    hover_name=df.columns[1],
    hover_data=df.columns,
    zoom=4,
    width=750,
    height=750
)

In [35]:
#fig.write_html("nome.html")

In [46]:
fig.write_html("codigo2.html")